Experiment is to classify given samples as positive or negative.
Colon Tumor data contains 62 samples collected from colon-cancer patients. Among them, 40 tumor biopsies are from tumors (labelled as "negative") and 22 normal (labelled as "positive") biopsies are from healthy parts of the colons of the same patients. Two thousand out of around 6500 genes were selected based on the confidence in the measured expression levels.


Two classifiers are used to do binary classification with different data specific variants

1) Nearest Neighbour
     a) with fixed K value
     b) with variable K value
     c) Class wise voting mechanism
2) SVM with linear kernel

For both the methods the preprocessing steps are same.
Normailzation technique is as below :
for every dimension among 2000 dimensions, "minimum" and "maximum" are found initially among 62 samples.
then every value is normailzed with below formula

f(x)=(x-min(dim_x))/(max(dim_x)-min(dim_x))

The experiments are done in the following procedure.

Step1 : Find the Minimum and maximum for each dimension among all 2000 dimensions

Step2 : Normalize the data

Step3 : Run classification



Experiment 1 :
Nearest Neighbour with class specific voting mechanism:

Standard k nearest neighbour technique uses voting mechanism among the k values.
But as our data is a bit biased i.e. positive class data size is 22 and negative class data size is 40, we are using a variant of voting mechanism for classification.
In this method we will take top k samples from +ve class and calculate average distance.
Again we will take top k samples from -ve class and claculate average distance.
Then we will assign the lable which is of less distance.
For all the sub experiments if the distance is same for +ve and -ve class then -ve class label is assigned as the number of -ve samples are more.(A prior belif)

Experiment 1a:
Here the K value is again class specific. i.e. we k value to take average is not same for both the classes.
K value is choosen as 50% value for +ve class(count 22) and 20% value for -ve class(count 40).
this heuristic is based on intution about the data.

Experiment 1b:
Here the K value is fixed to 5 for both the classes.

Experiment 2 :
Standard Nearest Neighbour:

Standard k nearest neighbour technique uses voting mechanism among the k values.
K value is fixed to 7.


Experiment 3 :
Here we use LibSVM along with linear Kerel. We can use polynomial or gaussian kernel. The data dimensions are high and number of samples are low we can work with Linear kernel itself.



In [1]:

import numpy as np
import math
import  random

min_array=np.zeros(2000);
max_array=np.zeros(2000);

pos_X=np.zeros([22,2000])
neg_X=np.zeros([40,2000])

pos_Y=np.zeros(22)
neg_Y=np.ones(40)

def findMinMax(ipath):
    count=0
    with open(ipath,'r') as fp:
        for line in fp:
            tokens=line.split(',')
            if(count==0):
                for i in range(0,2000):
                    min_array[i]=float(tokens[i].replace('\n',''));
                    max_array[i]=float(tokens[i].replace('\n',''));
                count=count+1;
            else:
                for i in range(0,2000):
                    cur_Val=float(tokens[i].replace('\n',''))
                    if(min_array[i]>cur_Val):min_array[i]=cur_Val;
                    if(max_array[i]<cur_Val):max_array[i]=cur_Val;


def prepareData2(ipath):
    pos_co=0;
    neg_co=0;
    with open(ipath,'r') as fp:
        for line in fp:
            tokens=line.split(',');
            tok2000=tokens[2000].strip().replace('\n','');
            if(tok2000=='positive'):
                addToArray(pos_co,tokens,'pos')
                pos_co=pos_co+1
            else:
                addToArray(neg_co,tokens,'neg')
                neg_co=neg_co+1



def addToArray(curr_line,tokens,val):
    if(val=='pos'):
     for i in range(0,2000):
        cur_val=normalize(i,float(tokens[i].strip()))
        pos_X[curr_line,i]=cur_val
    else:
     for i in range(0,2000):
        cur_val=normalize(i,float(tokens[i].strip()))
        neg_X[curr_line,i]=cur_val


def normalize(i,value):
    value=value-min_array[i];
    value=value/(max_array[i]-min_array[i]);
    return value



def euclideanDistance(instance1, instance2):
    distance = 0
    dims=len(instance1)
    for x in range(dims):
        distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)



#percent must be between 0 to 1
def generateRandomInstances(percent):
    pos_train_count=int(22*percent)
    neg_train_count=int(40*percent)
    rand_pos_train=random.sample(range(0,22),pos_train_count)
    rand_neg_train=random.sample(range(0,40),neg_train_count)
    rand_pos_test=(set(range(0,22))-set(rand_pos_train))
    rand_neg_test=(set(range(0,40))-set(rand_neg_train))
    return rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test


def getNeighbors(trX,teX, k):
	distances = []
	for x in range(len(trX)):
		dist = euclideanDistance(teX, trX[x])
		distances.append(dist)
	distances.sort()
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x])
	return neighbors

def classify(rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test,percent,varKFlag):
    if(varKFlag==1):pos_k,neg_k=calculateK(percent)
    else:
        pos_k=5
        neg_k=5
    pos_labels=[]
    pos_test_samples=[pos_X[i] for i in rand_pos_test]
    for pos_sample in pos_test_samples:
        pos_neighbors=getNeighbors(pos_X[rand_pos_train],pos_sample,pos_k)
        neg_neighbors=getNeighbors(neg_X[rand_neg_train],pos_sample,neg_k)
        pos_labels.append(calcLabel(pos_neighbors,neg_neighbors))

    neg_labels=[]
    neg_test_samples=[neg_X[i] for i in rand_neg_test]
    for neg_sample in neg_test_samples:
        pos_neighbors=getNeighbors(pos_X[rand_pos_train],neg_sample,pos_k)
        neg_neighbors=getNeighbors(neg_X[rand_neg_train],neg_sample,neg_k)
        neg_labels.append(calcLabel(pos_neighbors,neg_neighbors))
    return pos_labels,neg_labels



def calcLabel(pos_neighbors,neg_neighbors):
    pos_k=len(pos_neighbors)
    neg_k=len(neg_neighbors)
    pos_dist=0
    neg_dist=0
    for neighbor in pos_neighbors:
        pos_dist+=neighbor
    pos_dist=pos_dist/pos_k;
    for neighbor in neg_neighbors:
        neg_dist+=neighbor
    neg_dist=neg_dist/neg_k;
    if(pos_dist<neg_dist):
        return 0
    elif(pos_dist>neg_dist):
        return 1
    else: return 1 #prior belif


def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if int(testSet[x]) is int(predictions[x]):
			correct += 1
	return (correct/float(len(testSet))) * 100.0

def calculateK(percent):
    pos_train_count=int(22*percent)
    neg_train_count=int(40*percent)
    pos_k=int(pos_train_count*0.5)
    neg_k=int(neg_train_count*0.2)
    return pos_k,neg_k


def NN():
    ipath='/home/kushwanth/ClassificationColonTumor/ColonTumor/colonTumor.data'
    findMinMax(ipath);
    prepareData2(ipath);
    percents=[0.3,0.4,0.5,0.6]
    print "with random K value : \n"
    #print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    for percent in percents:
        rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test=generateRandomInstances(percent)
        pos_labels,neg_labels=classify(rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test,percent,1)
        pos_labels=np.asarray(pos_labels)
        neg_labels=np.asarray(neg_labels)
        pred_labels=np.concatenate([pos_labels,neg_labels],axis=0)
        curr_pos_y=np.zeros(len(rand_pos_test))
        curr_neg_y=np.ones(len(rand_neg_test))
        actual_labels=np.concatenate([curr_pos_y,curr_neg_y],axis=0)
        print "for "+str(percent*100)+"% training data, accuracy is : " +str(getAccuracy(actual_labels,pred_labels))
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    print "with fixed k value : \n"
    #print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    for percent in percents:
        rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test=generateRandomInstances(percent)
        pos_labels,neg_labels=classify(rand_pos_train,rand_neg_train,rand_pos_test,rand_neg_test,percent,0)
        pos_labels=np.asarray(pos_labels)
        neg_labels=np.asarray(neg_labels)
        pred_labels=np.concatenate([pos_labels,neg_labels],axis=0)
        curr_pos_y=np.zeros(len(rand_pos_test))
        curr_neg_y=np.ones(len(rand_neg_test))
        actual_labels=np.concatenate([curr_pos_y,curr_neg_y],axis=0)
        print "for "+str(percent*100)+"% training data, accuracy is : " +str(getAccuracy(actual_labels,pred_labels))



NN();



with random K value : 

for 30.0% training data, accuracy is : 52.2727272727
for 40.0% training data, accuracy is : 57.8947368421
for 50.0% training data, accuracy is : 64.5161290323
for 60.0% training data, accuracy is : 60.0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++
with fixed k value : 

for 30.0% training data, accuracy is : 63.6363636364
for 40.0% training data, accuracy is : 52.6315789474
for 50.0% training data, accuracy is : 61.2903225806
for 60.0% training data, accuracy is : 60.0


In [2]:
#this  file prepares the colon tumor data in libsvm format




#read data line by line
#lines in colontumor data is of format
'''
 <feature1>,<feature2>,<feature3>.......,<feature2000>,<label>
'''

#required libsvm format
'''
 Note 0 for positive 1 for negative
 <label> 1:<feature1> 2:<feature2> 3:<feature3>.......2000:<feature2000>
'''

'''
Contains 62 samples collected from colon-cancer patients.
Among them, 40 tumor biopsies are from tumors (labelled as "negative")
and 22 normal (labelled as "positive") biopsies are from healthy parts of the colons of the same patients.
Two thousand out of around 6500 genes were selected based on the confidence in the measured expression levels.

we will divide 70:30 as train:test
train
40*0.7=28
22*0.7=15

test
40-28=12
22-15=7
'''

import numpy as np


#ipath is input colon.data path
#opath is folder where to store the train and test data
#train_persent is persent of train range from 0.1 to 0.8 usually

import numpy as np
import os
from svmutil import *


min_array=np.zeros(2000);
max_array=np.zeros(2000);


def findMinMax(ipath):
    count=0
    with open(ipath,'r') as fp:
        for line in fp:
            tokens=line.split(',')
            if(count==0):
                for i in range(0,2000):
                    min_array[i]=float(tokens[i]);
                    max_array[i]=float(tokens[i]);
                count=count+1;
                continue;
            for i in range(0,2000):
                cur_Val=float(tokens[i])
                if(min_array[i]>cur_Val):min_array[i]=cur_Val;
                if(max_array[i]<cur_Val):max_array[i]=cur_Val;


def prepareData(ipath,train_persent):
    pos_count=0;
    neg_count=0;
    t_pos_count=int(22*train_persent);
    print t_pos_count
    t_neg_count=int(40*train_persent);
    print t_neg_count
    with open(ipath,'r') as fp:
        for line in fp:
            tokens=line.split(',');
            '''
            if(len(tokens)!=2001):
                print count
            '''
            #print len(tokens)
            curr_line=''
            #print tokens[2000]
            tok2000=tokens[2000].strip();
            if(tok2000=='positive'):
                curr_line='0';
                pos_count=pos_count+1;
                if(pos_count<=t_pos_count):
                    path='/home/kushwanth/ClassificationColonTumor/train.data';
                    addToFile(curr_line,path,tokens);
                else:
                    path='/home/kushwanth/ClassificationColonTumor/test.data';
                    addToFile(curr_line,path,tokens);
            else:
                curr_line='1';
                neg_count=neg_count+1;
                if(neg_count<=t_neg_count):
                    path='/home/kushwanth/ClassificationColonTumor/train.data';
                    addToFile(curr_line,path,tokens);
                else:
                    path='/home/kushwanth/ClassificationColonTumor/test.data';
                    addToFile(curr_line,path,tokens);



def prepareData2(ipath,opath):
    with open(ipath,'r') as fp:
        for line in fp:
            tokens=line.split(',');
            tok2000=tokens[2000].strip();
            if(tok2000=='positive'):
                curr_line='0';
                addToFile(curr_line,os.path.join(opath,'pos'),tokens)
            else:
                curr_line='1';
                addToFile(curr_line,os.path.join(opath,'neg'),tokens)


def addToFile(curr_line,path_new,tokens):
    for i in range(0,2000):
        j=i+1
        cur_val=normalize(i,float(tokens[i].strip()))
        curr_line=curr_line+' '+str(j)+':'+str(cur_val)
    curr_line=curr_line+'\n'
    with open(path_new,'a') as ofp:
        ofp.write(curr_line)

def normalize(i,value):
    value=value-min_array[i];
    value=value/(max_array[i]-min_array[i]);
    return value


def runSVM(opath,kernal):
    y_pos, x_pos = svm_read_problem(os.path.join(opath,'pos'));
    y_neg, x_neg = svm_read_problem(os.path.join(opath,'neg'));
    train_persent=0.3;
    for i in range(1,6):
        t_pos_count=int(22*train_persent);
        t_neg_count=int(40*train_persent);
        print "**********start***************"
        print "training data percent : " +str(train_persent*100)
        print "current # of training positive count "+str(len(y_pos[0:t_pos_count]))
        print "current # of training negative count "+str(len(y_neg[0:t_neg_count]))
        curr_train_y=y_pos[0:t_pos_count]+y_neg[0:t_neg_count];
        curr_train_x=x_pos[0:t_pos_count]+x_neg[0:t_neg_count];
        m = svm_train(curr_train_y,curr_train_x,kernal)
        print "current # of test positive count "+str(len(y_pos[t_pos_count:len(y_pos)]))
        print "current # of test negative count "+str(len(y_neg[t_neg_count:len(y_neg)]))
        curr_test_y=y_pos[t_pos_count:len(y_pos)]+y_neg[t_neg_count:len(y_neg)];
        curr_test_x=x_pos[t_pos_count:len(x_pos)]+x_neg[t_neg_count:len(x_neg)];
        p_label, p_acc, p_val = svm_predict(curr_test_y,curr_test_x, m)
        train_persent=train_persent+0.1
        print "================================="


def cleanPrev():
    cur_path=os.path.join(opath,'pos')
    if(os.path.isfile(cur_path)):os.remove(cur_path)
    cur_path=os.path.join(opath,'neg')
    if(os.path.isfile(cur_path)):os.remove(cur_path)

opath='/home/kushwanth/ClassificationColonTumor/libsvm_format'
ipath='/home/kushwanth/ClassificationColonTumor/ColonTumor/colonTumor.data'
findMinMax(ipath)
#print min_array
#print max_array
cleanPrev()
prepareData2(ipath,opath);
print "0 -- linear: u'*v"
kernal='-t 0'
runSVM(opath,kernal);

#prepareData(ipath,0.7);

0 -- linear: u'*v
**********start***************
training data percent : 30.0
current # of training positive count 6
current # of training negative count 12
current # of test positive count 16
current # of test negative count 28
Accuracy = 86.3636% (38/44) (classification)
**********start***************
training data percent : 40.0
current # of training positive count 8
current # of training negative count 16
current # of test positive count 14
current # of test negative count 24
Accuracy = 92.1053% (35/38) (classification)
**********start***************
training data percent : 50.0
current # of training positive count 11
current # of training negative count 20
current # of test positive count 11
current # of test negative count 20
Accuracy = 83.871% (26/31) (classification)
**********start***************
training data percent : 60.0
current # of training positive count 13
current # of training negative count 24
current # of test positive count 9
current # of test negative count 16
Acc